<a href="https://colab.research.google.com/github/ptonydb/map-gen/blob/main/Map_Generation_With_ASP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/adamsumm/AI_Minecraft_Assignments/master/ASPHouseGeneration/world_gen.lp
!wget https://github.com/potassco/clingo/releases/download/v5.4.0/clingo-5.4.0-linux-x86_64.tar.gz
!tar -xf clingo-5.4.0-linux-x86_64.tar.gz
!mv clingo-5.4.0-linux-x86_64/* .
!touch house_gen.lp
!cat world_gen.lp

--2020-11-04 06:01:21--  https://raw.githubusercontent.com/adamsumm/AI_Minecraft_Assignments/master/ASPHouseGeneration/world_gen.lp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3520 (3.4K) [text/plain]
Saving to: ‘world_gen.lp.3’

world_gen.lp.3      100%[===================>]   3.44K  --.-KB/s    in 0s      

2020-11-04 06:01:21 (46.2 MB/s) - ‘world_gen.lp.3’ saved [3520/3520]

--2020-11-04 06:01:21--  https://github.com/potassco/clingo/releases/download/v5.4.0/clingo-5.4.0-linux-x86_64.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-production-release-asset-2e65be.s3.amazonaws.com/58459878/14b79000-c7f5-11e

In [ ]:
import json
import collections
import subprocess
import random
import sys


def solve(args):
    """Run clingo with the provided argument list and return the parsed JSON result."""

    print_args = ['clingo'] + list(args) + [' | tr [:space:] \\\\n | sort ']
    args = ['./clingo', '--outf=2'] + args + ["--sign-def=rnd","--seed="+str(random.randint(0,1<<30))]
    
    with subprocess.Popen(
        ' '.join(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True
    ) as clingo:
        outb, err = clingo.communicate()
    if err:
        print(err)
    print(outb)
    out = outb.decode("utf-8")
    with open('dump.lp', 'w') as outfile:
        result = json.loads(out)
        witness = result['Call'][0]['Witnesses'][-1]['Value']
        for atom in sorted(witness):
            outfile.write(atom + '\n')
    
    return parse_json_result(out)


def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""

    result = json.loads(out)

    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0

    witness = result['Call'][0]['Witnesses'][0]['Value']

    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key

    preds = collections.defaultdict(set)
    env = identitydefaultdict()

    for atom in witness:
        if '(' in atom:
            left = atom.index('(')
            functor = atom[:left]
            arg_string = atom[left:]
            try:
                preds[functor].add( eval(arg_string, env) )
            except TypeError:
                pass # at least we tried...

        else:
            preds[atom] = True
    return dict(preds)


def pretty_print(world):
    import functools
    width = functools.reduce(lambda x,y: max(x,y),world['width'])
    height = functools.reduce(lambda x,y: max(x,y),world['height'])
    grid = [[ ' ' for w in range(width)] for h in range(height)]

    legend = {'grass':'🌿',
    'flower':'🌼',
    'bridge':'🌉',
    'water':'🌊',
    'tree':'🌲',
    'door':'🚪',
    'wall':'🗄️',
    'bed':'🛏️',
    'floor':'🔲'}


    for t in world['terrain']:
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
    for t in world.get('construction',[]):
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
        
    return ('\n'.join([''.join(r) for r in grid]))

In [ ]:
%%time

width = 12
height = 12

world = solve(['world_gen.lp','-c','max_width={}'.format(width),'-c','max_height={}'.format(height),'-t','2'])

print(pretty_print(world))


b'{\n  "Solver": "clingo version 5.4.0",\n  "Input": [\n    "world_gen.lp"\n  ],\n  "Call": [\n    {\n      "Witnesses": [\n        {\n          "Value": [\n            "width(1)", "width(2)", "width(3)", "width(4)", "width(5)", "width(6)", "width(7)", "width(8)", "width(9)", "width(10)", "width(11)", "width(12)", "height(1)", "height(2)", "height(3)", "height(4)", "height(5)", "height(6)", "height(7)", "height(8)", "height(9)", "height(10)", "height(11)", "height(12)", "terrain_type(grass)", "terrain_type(tree)", "terrain_type(flower)", "terrain_type(water)", "terrain_type(bridge)", "terrain(1,2,grass)", "terrain(2,2,grass)", "terrain(5,2,grass)", "terrain(6,2,grass)", "terrain(12,2,grass)", "terrain(1,3,grass)", "terrain(6,3,grass)", "terrain(12,3,grass)", "terrain(1,4,grass)", "terrain(6,4,grass)", "terrain(11,4,grass)", "terrain(12,4,grass)", "terrain(1,5,grass)", "terrain(2,5,grass)", "terrain(9,5,grass)", "terrain(10,5,grass)", "terrain(11,5,grass)", "terrain(12,5,grass)", "terra

In [ ]:
def write_to_file(text,filename):
  with open(filename,'w') as outfile:
    outfile.write(text)

In [ ]:
%%time

inline = True 

if inline:
  world_gen = '''
    #const max_width=12.
    #const max_height=12.
    construction_type(door;wall;floor;bed).


    {construction(XX,YY,CONSTR_TYPE) : construction_type(CONSTR_TYPE)} 1 :- width(XX), height(YY),
          not terrain(XX,YY,tree), not terrain(XX,YY,bridge), not terrain(XX,YY,water).
  

    h_immediate_neighbors(XX,YY,CONSTR_TYPE, COUNT) :- 
	        COUNT =  {construction(XX,YY-1,CONSTR_TYPE);
					construction(XX-1,YY,CONSTR_TYPE);
					construction(XX+1,YY,CONSTR_TYPE);
					construction(XX,YY+1,CONSTR_TYPE)},
					construction(XX,YY,_),					
					construction_type(CONSTR_TYPE).

    h_neighbors(XX,YY,CONSTR_TYPE, COUNT) :- 
	        COUNT =  {construction(XX,YY-1,CONSTR_TYPE);
					construction(XX-1,YY-1,CONSTR_TYPE);
					construction(XX+1,YY-1,CONSTR_TYPE);
          construction(XX+1,YY,CONSTR_TYPE);
					construction(XX+1,YY+1,CONSTR_TYPE);
					construction(XX-1,YY,CONSTR_TYPE);
					construction(XX-1,YY+1,CONSTR_TYPE);
					construction(XX,YY+1,CONSTR_TYPE)},
					construction(XX,YY,_),					
					construction_type(CONSTR_TYPE).


    %IS CONSTRUCTION TYPE
    neighbors_are_construction(XX,YY) :- construction(XX-1,YY,_),construction(XX+1,YY,_),construction(XX,YY+1,_),construction(XX,YY-1,_).
    %IS CONSTRUCTION TYPE


    %EDGE DOORS
    :- construction(1,YY,door).
    :- construction(max_width,YY,door).
    :- construction(XX,1,door).
    :- construction(XX,max_height,door).

    %OPEN TO ONLY FLOWERS AND GRASS
    :- construction(XX,YY,door), immediate_neighbors(XX,YY,water,COUNT), COUNT > 0.
    :- construction(XX,YY,door), immediate_neighbors(XX,YY,bridge,COUNT), COUNT > 0.
    :- construction(XX,YY,door), immediate_neighbors(XX,YY,tree,COUNT), COUNT > 0.
    
    %A door with 3 walls as immediate neighbors is a door that leads nowhere
    :- construction(XX,YY,door), h_immediate_neighbors(XX,YY,wall,COUNT), COUNT > 2.

    %create walls parallel to door
    surrounded_by_walls(XX,YY) :- construction(XX+1,YY,wall), construction(XX-1,YY,wall).
    surrounded_by_walls(XX,YY) :- construction(XX,YY-1,wall), construction(XX,YY+1,wall).
    :- construction(XX,YY,door), not surrounded_by_walls(XX,YY).

    %wall thinning
    :- construction(XX,YY,wall), h_immediate_neighbors(XX,YY,wall,W_COUNT),
       construction(XX,YY,wall), h_immediate_neighbors(XX,YY,door,D_COUNT),W_COUNT + D_COUNT != 2.
    :- construction(XX,YY,wall), h_neighbors(XX,YY,wall,W_COUNT),
       construction(XX,YY,wall), h_neighbors(XX,YY,door,D_COUNT),W_COUNT + D_COUNT > 4.
    :- construction(XX,YY,wall), h_neighbors(XX,YY,floor,COUNT), COUNT < 1.

    %no outdoor snoozing
    :- construction(XX,YY,bed), not neighbors_are_construction(XX,YY).
    %let's not sleep so close to one another
    :- construction(XX,YY,bed), h_neighbors(XX,YY,bed,COUNT), COUNT > 1.
    %runaway tiles
    :- construction(XX,YY,floor), not neighbors_are_construction(XX,YY).
    %at least one of each construction type
    :- {construction(XX,YY,CONSTR_TYPE)} 0, construction_type(CONSTR_TYPE).
    

  '''
  write_to_file(world_gen,'house_gen.lp')

else:
  !rm house_gen.lp
  path_to_your_file = 'http://this_is_a_dummy_address.com/you_need_to_change_it/house_gen.lp'
  !wget {path_to_your_file}



width = 12
height = 12
world = solve(['world_gen.lp','house_gen.lp','-c','max_width={}'.format(width),'-c','max_height={}'.format(height),'-t','2'])

print(pretty_print(world))

b'{\n  "Solver": "clingo version 5.4.0",\n  "Input": [\n    "world_gen.lp","house_gen.lp"\n  ],\n  "Call": [\n    {\n      "Witnesses": [\n        {\n          "Value": [\n            "construction_type(door)", "construction_type(wall)", "construction_type(floor)", "construction_type(bed)", "height(1)", "height(2)", "height(3)", "height(4)", "height(5)", "height(6)", "height(7)", "height(8)", "height(9)", "height(10)", "height(11)", "height(12)", "width(1)", "width(2)", "width(3)", "width(4)", "width(5)", "width(6)", "width(7)", "width(8)", "width(9)", "width(10)", "width(11)", "width(12)", "terrain_type(grass)", "terrain_type(tree)", "terrain_type(flower)", "terrain_type(water)", "terrain_type(bridge)", "terrain(1,2,grass)", "terrain(2,2,grass)", "terrain(6,2,grass)", "terrain(7,2,grass)", "terrain(8,2,grass)", "terrain(9,2,grass)", "terrain(10,2,grass)", "terrain(11,2,grass)", "terrain(1,3,grass)", "terrain(8,3,grass)", "terrain(9,3,grass)", "terrain(10,3,grass)", "terrain(11,3,grass

In [ ]:
!./clingo world_gen.lp house_gen.lp -c max_width=12 -c max_height=12

clingo version 5.4.0
Reading from world_gen.lp ...
Solving...
Answer: 1
construction_type(door) construction_type(wall) construction_type(floor) construction_type(bed) height(1) height(2) height(3) height(4) height(5) height(6) height(7) height(8) height(9) height(10) height(11) height(12) width(1) width(2) width(3) width(4) width(5) width(6) width(7) width(8) width(9) width(10) width(11) width(12) terrain_type(grass) terrain_type(tree) terrain_type(flower) terrain_type(water) terrain_type(bridge) terrain(1,2,grass) terrain(3,2,grass) terrain(4,2,grass) terrain(5,2,grass) terrain(6,2,grass) terrain(12,2,grass) terrain(1,3,grass) terrain(2,3,grass) terrain(3,3,grass) terrain(4,3,grass) terrain(5,3,grass) terrain(6,3,grass) terrain(12,3,grass) terrain(1,4,grass) terrain(2,4,grass) terrain(3,4,grass) terrain(5,4,grass) terrain(6,4,grass) terrain(7,4,grass) terrain(11,4,grass) terrain(12,4,grass) terrain(1,5,grass) terrain(7,5,grass) terrain(11,5,grass) terrain(12,5,grass) terrain(1,6,gras